In [1]:
#@Imports
import sys
import os
path = os.path.abspath('..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

import numpy as np
import matplotlib.pyplot as plt

from IPython.display import clear_output
from retry import retry

from language_models.ollama_model import OllamaLanguageModel

from components.components import compute_desire_for_gamble, multiple_choice_preferences, compute_distribution_of_desire_for_gamble, compute_distribution_of_desire_for_gamble_binaries

In [2]:
system_message = ("This is an agent based model. "
  f"The goal of the LLM to to play characters in a game, and act as humanlike as possible. "
  "Ideally, human observers should not be able to tell the difference between the LLM and a human player. "
)

model = OllamaLanguageModel(
"llama3:70b", system_message=system_message, streaming=False
)

In [4]:
expected_values = []
attitudes = []
for i in range(2):
  # Simulate decision making for each player.
  Good_value = np.random.choice(np.arange(10))
  Bad_value = np.random.choice(np.arange(10))
  P_w = np.random.choice(np.arange(10) / 10)

  # Compute the expected value of the gamble.
  expected_value = P_w * Good_value + (1 - P_w) * -1*Bad_value
  gamble_input = f"Probability of Winning (P_w): {P_w}, Positive Value on Win (G): {Good_value}, Negative Value on Loss (B): {-Bad_value}"
  # Compute affective LLM value estimate
  llm_value = compute_desire_for_gamble(model, gamble_input)
  expected_values.append(expected_value)
  attitudes.append(llm_value)
  takepass =  multiple_choice_preferences(model, None, gamble_input, llm_value)
  dist = compute_distribution_of_desire_for_gamble(model, gamble_input)
  print(gamble_input)
  print("all at once: ", dist)
  original_probabilities, softmax_probabilities = compute_distribution_of_desire_for_gamble_binaries(model, gamble_input)
  print("yes/no outputs: ", original_probabilities)
  print("softmax of yes/ne: ", softmax_probabilities)

  # Print trial outputs
  #clear_output(wait = True)
  print(f"╔════════════════╗")
  print(f"║   {'Trial ' + str(i+1)+':':<10}   ║")
  print(f"╠════════════════╣")
  print(f"║  EV:    {round(expected_value, 2):<6} ║")
  print(f"║  Value: {round(llm_value, 2):<6} ║")
  print(f"║  Action: {takepass} ║")
  print(f"╚════════════════╝")



Probability of Winning (P_w): 0.6, Positive Value on Win (G): 9, Negative Value on Loss (B): -3
all at once:  0.01, 0.02, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.40, 0.50


In [ ]:
%autoreload 2
from components.prospect_theory import curve_fit, plot_curve

outcomes = curve_fit(expected_values, attitudes)
plot_curve(*outcomes, expected_values, attitudes, title = "LlaMA3: Risky gamble value estimates")